In [1]:
import glob
import time
import os
import pandas as pd
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
from argparse import ArgumentParser, Namespace
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from itertools import chain
from tqdm import tqdm
import copy
import shutil
import pickle

In [2]:
es_url = 'http://ckg07:9200'
es_index = 'wikidatadwd-augmented'

# Input Paths

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-canonical/
train_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/train1-canonical"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-canonical/
dev_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/dev-canonical"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/ground_truth/Xinting_GT_csv
ground_truth_files = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/round4_gt_with_labels"


# OUTPUT PATHS
output_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker"
train_output_path = f'{output_path}/train1-output/tbl-pipeline-nn'
dev_output_path = f'{output_path}/dev-output/tbl-pipeline-nn'

# increase version to create a new folder for an experiment
PREVIOUS_VERSION = "v18"
VERSION = "v19"

train_candidate_path = f'{train_output_path}/{VERSION}/candidates'
train_features1_path = f'{train_output_path}/{VERSION}/features1'
train_features2_path = f'{train_output_path}/{VERSION}/features2'
train_features3_path = f'{train_output_path}/{VERSION}/features3'
train_feature_path = f'{train_output_path}/{VERSION}/features'
train_missing_candidates_path = f'{train_output_path}/{VERSION}/train_missing_candidates_path'

dev_candidate_path = f'{dev_output_path}/{VERSION}/candidates'
dev_feature_path = f'{dev_output_path}/{VERSION}/features'
dev_features1_path = f'{dev_output_path}/{VERSION}/features1'
dev_features2_path = f'{dev_output_path}/{VERSION}/features2'
dev_features3_path = f'{dev_output_path}/{VERSION}/features3'
dev_output_predictions = f'{dev_output_path}/{VERSION}/dev_predictions'
dev_predictions_top_k = f'{dev_output_path}/{VERSION}/dev_predictions_top_k'
dev_colorized_path = f'{dev_output_path}/{VERSION}/dev_predictions_colorized'
dev_metrics_path = f'{dev_output_path}/{VERSION}/dev_predictions_metrics'
dev_missing_candidates_path = f'{dev_output_path}/{VERSION}/dev_missing_candidates_path'

aux_field = 'graph_embedding_complex,class_count,property_count,context'


train_prop_count = f'{train_output_path}/{VERSION}/train_prop_count' 
train_class_count = f'{train_output_path}/{VERSION}/train_class_count'
train_context_path = f'{train_output_path}/{VERSION}/train_context'
train_graph_embedding = f'{train_output_path}/{VERSION}/train_graph_embedding'
train_context_property_path = f'{train_output_path}/{VERSION}/train_context_properties'

dev_prop_count = f'{dev_output_path}/{VERSION}/dev_prop_count'
dev_class_count = f'{dev_output_path}/{VERSION}/dev_class_count'
dev_context_path = f'{dev_output_path}/{VERSION}/dev_context'
dev_graph_embedding = f'{dev_output_path}/{VERSION}/dev_graph_embedding'
dev_context_property_path = f'{dev_output_path}/{VERSION}/dev_context_properties'

temp_dir = f'{output_path}/temp'

pseudo_gt_model = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/pseudo-gt-nn/v19/saved_models/epoch_1_loss_0.534353494644165_top1_0.7883487007544007.pth'
pseudo_gt_min_max_scaler_path = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/temp/training_data/normalization_factor.pkl'

tl_log_file =f'{temp_dir}/tl_log.txt'

pos_output = f'{temp_dir}/training_data/tl_pipeline_pos_features.pkl'
neg_output = f'{temp_dir}/training_data/tl_pipeline_neg_features.pkl'
min_max_scaler_path = f'{temp_dir}/training_data/tl_pipeline_normalization_factor.pkl'

final_score_column = 'siamese_prediction'
threshold = final_score_column+":median"

model_save_path = f'{dev_output_path}/{VERSION}/saved_models'
best_model_path = ''

copy_candidates_from_previous_version = False
copy_features1_from_previous_version = False

In [3]:
!mkdir -p "$temp_dir"

!mkdir -p "$train_prop_count"
!mkdir -p "$dev_prop_count"
!mkdir -p "$train_class_count"
!mkdir -p "$dev_class_count"
!mkdir -p "$train_graph_embedding"
!mkdir -p "$dev_graph_embedding"
!mkdir -p "$train_context_path"
!mkdir -p "$dev_context_path"

!mkdir -p "$train_candidate_path"
!mkdir -p "$dev_candidate_path"

!mkdir -p "$train_feature_path"
!mkdir -p "$train_features1_path"
!mkdir -p "$train_features2_path"
!mkdir -p "$train_features3_path"
!mkdir -p "$dev_feature_path"
!mkdir -p "$dev_features1_path"
!mkdir -p "$dev_features2_path"
!mkdir -p "$dev_features3_path"

!mkdir -p "$temp_dir/training_data"
!mkdir -p "$dev_output_predictions"
!mkdir -p "$model_save_path"
!mkdir -p "$dev_predictions_top_k"
!mkdir -p "$dev_colorized_path"
!mkdir -p "$dev_metrics_path"
!mkdir -p "$dev_missing_candidates_path"
!mkdir -p "$train_missing_candidates_path"
!mkdir -p "$train_context_property_path"
!mkdir -p "$dev_context_property_path"

In [ ]:
if copy_candidates_from_previous_version:
    !cp $dev_output_path/$PREVIOUS_VERSION/candidates/*csv $dev_output_path/$VERSION/candidates
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_prop_count/* $dev_output_path/$VERSION/dev_prop_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_class_count/* $dev_output_path/$VERSION/dev_class_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_context/* $dev_output_path/$VERSION/dev_context
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_graph_embedding/* $dev_output_path/$VERSION/dev_graph_embedding
    
    !cp $train_output_path/$PREVIOUS_VERSION/candidates/*csv $train_output_path/$VERSION/candidates
    !cp $train_output_path/$PREVIOUS_VERSION/train_prop_count/* $train_output_path/$VERSION/train_prop_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_class_count/* $train_output_path/$VERSION/train_class_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_context/* $train_output_path/$VERSION/train_context
    !cp $train_output_path/$PREVIOUS_VERSION/train_graph_embedding/* $train_output_path/$VERSION/train_graph_embedding

In [4]:
pseudo_gt_features = ["monge_elkan","monge_elkan_aliases","jaro_winkler","levenshtein","singleton","pgr_rts","context_score","smc_class_score","smc_property_score"]

## Candidate Generation

In [ ]:
def candidate_generation(path, gt_path, output_path, class_count_path, prop_count_path, context_path, graph_embedding):
    file_list = glob.glob(path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        st = time.time()
        filename = file.split('/')[-1]
        gt_file = f"{ground_truth_files}/{filename}"
        output_file = f"{output_path}/{filename}"
        
        !tl --log-file $tl_log_file clean -c label -o label_clean "$file" / \
        --url $es_url --index $es_index \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        --url $es_url --index $es_index \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        / get-ngram-matches -c label_clean  \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" \
        / ground-truth-labeler --gt-file "$gt_file" > "$output_file"

        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = f"{class_count_path}/{filename.strip('.csv')}_class_count.tsv"
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = f"{prop_count_path}/{filename.strip('.csv')}_prop_count.tsv"
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            elif field == 'context':
                context_file = f"{context_path}/{filename.strip('.csv')}_context.tsv"
                aux_df.to_csv(context_file, sep='\t', index=False)
            else:
                graph_embedding_file = f"{graph_embedding}/{filename.strip('.csv')}_graph_embedding_complex.tsv"
                aux_df.to_csv(graph_embedding_file, sep='\t', index=False)
        

In [ ]:
if not copy_candidates_from_previous_version:
    candidate_generation(train_path, ground_truth_files, train_candidate_path, train_class_count, train_prop_count, train_context_path,train_graph_embedding)

In [ ]:
if not copy_candidates_from_previous_version:
    candidate_generation(dev_path, ground_truth_files, dev_candidate_path, dev_class_count, dev_prop_count, dev_context_path, dev_graph_embedding)

## Feature Generation

In [ ]:
def features_1(candidate_dir, output_path, context_path, class_count_dir, property_count_dir, pseudo_gt_model, pseudo_gt_min_max_scaler_path):
    file_list = glob.glob(candidate_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        
        context_file = f"{context_path}/{filename[:-4]}_context.tsv"
        class_count_file = f"{class_count_dir}/{filename[:-4]}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename[:-4]}_prop_count.tsv"
        feature_str =  ",".join(pseudo_gt_features)

        !tl --log-file $tl_log_file string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan --threshold 0.85 $file \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases --threshold 0.85  \
            / string-similarity -i --method jaro_winkler -o jaro_winkler --threshold 0.85  \
            / string-similarity -i --method levenshtein -o levenshtein --threshold 0.85  \
            / create-singleton-feature -o singleton \
            / pick-hc-candidates -o ignore_candidate -s monge_elkan,monge_elkan_aliases,jaro_winkler,levenshtein \
            / context-match --debug --context-file $context_file --ignore-column-name ignore_candidate -o context_score \
            / kth-percentile -c context_score -o kth_percenter --ignore-column ignore_candidate --k-percentile 0.75  --minimum-cells 10 \
            / pgt-semantic-tf-idf \
            -o smc_class_score \
            --pagerank-column pagerank \
            --retrieval-score-column retrieval_score \
            --feature-file "$class_count_file" \
            --feature-name class_count \
            --high-confidence-column kth_percenter \
            / pgt-semantic-tf-idf \
            -o smc_property_score \
            --pagerank-column pagerank \
            --retrieval-score-column retrieval_score \
            --feature-file "$property_count_file" \
            --feature-name property_count \
            --high-confidence-column kth_percenter \
            / predict-using-model -o pseudo_gt_prediction \
            --features $feature_str \
            --ranking-model $pseudo_gt_model \
            --normalization-factor $pseudo_gt_min_max_scaler_path \
            / create-pseudo-gt -o pseudo_gt \
            --column-thresholds pseudo_gt_prediction:mean \
            --filter smc_class_score:0 > $output_file

In [ ]:
if not copy_features1_from_previous_version:
    features_1(train_candidate_path, train_features1_path, train_context_path, train_class_count, train_prop_count,pseudo_gt_model, pseudo_gt_min_max_scaler_path)
else:
    !cp $train_output_path/$PREVIOUS_VERSION/features1/*csv $train_output_path/$VERSION/features1

In [ ]:
if not copy_features1_from_previous_version:
    features_1(dev_candidate_path, dev_features1_path, dev_context_path, dev_class_count, dev_prop_count,pseudo_gt_model, pseudo_gt_min_max_scaler_path)
else:
    !cp $dev_output_path/$PREVIOUS_VERSION/features1/*csv $dev_output_path/$VERSION/features1

In [ ]:
def features_2(features1_dir, output_path, context_path, context_property_path):
    file_list = glob.glob(features1_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        context_file = f"{context_path}/{filename[:-4]}_context.tsv"
        context_property_file = f"{context_property_path}/{filename[:-4]}_context_properties.csv"

        !tl --log-file $tl_log_file context-match $file -o context_score_2 \
        --context-file $context_file --save-property-scores-path $context_property_file \
        --pseudo-gt-column-name pseudo_gt > $output_file
#         / context-match --context-file $context_file -o context_score_3 \
#         --use-saved-property-scores $context_property_file > $output_file

In [ ]:
features_2(train_features1_path, train_features2_path, train_context_path, train_context_property_path)

In [ ]:
features_2(dev_features1_path, dev_features2_path, dev_context_path, dev_context_property_path)

In [ ]:
def features_2_1(features2_dir, output_path, context_path, context_property_path):
    file_list = glob.glob(features2_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        context_file = f"{context_path}/{filename[:-4]}_context.tsv"
        context_property_file = f"{context_property_path}/{filename[:-4]}_context_properties.csv"
        if os.path.exists(output_file) or filename == '0LZ0M8W4.csv':
            continue

        !tl --log-file $tl_log_file context-match $file --context-file $context_file -o context_score_3 \
        --use-saved-property-scores $context_property_file > $output_file

In [ ]:
features_2_1(train_features2_path, train_features3_path, train_context_path, train_context_property_path)

In [ ]:
features_2_1(dev_features2_path, dev_features3_path, dev_context_path, dev_context_property_path)

In [5]:
def features_3(features3_dir, output_path, ge_path, class_count_dir, property_count_dir):
    file_list = glob.glob(features3_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        embedding_file = f"{ge_path}/{filename[:-4]}_graph_embedding_complex.tsv"
        class_count_file = f"{class_count_dir}/{filename[:-4]}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename[:-4]}_prop_count.tsv"

        !tl mosaic-features -c kg_labels --num-char --num-tokens $file \
        / score-using-embedding \
        --column-vector-strategy centroid-of-lof \
        --lof-strategy pseudo-gt \
        -o pgt_centroid_score \
        --embedding-file $embedding_file \
        / compute-tf-idf  \
        --feature-file $class_count_file \
        --feature-name class_count \
        --singleton-column pseudo_gt \
        -o pgt_class_count_tf_idf_score \
        / compute-tf-idf \
        --feature-file $property_count_file \
        --feature-name property_count \
        --singleton-column pseudo_gt \
        -o pgt_property_count_tf_idf_score > $output_file

In [6]:
features_3(train_features3_path, train_feature_path, train_graph_embedding, train_class_count, train_prop_count)

0it [00:00, ?it/s]

mosaic-features Time: 0.005096912384033203s
Qnodes to lookup: 1534
Qnodes from file: 1516
Outlier removal generates 4 lof-voted candidates
score-using-embedding Time: 1.3668348789215088s
compute-tf-idf-class_count Time: 2.041016101837158s
compute-tf-idf-property_count Time: 2.2113211154937744s


1it [00:10, 10.40s/it]

mosaic-features Time: 0.006936073303222656s
Qnodes to lookup: 926
Qnodes from file: 915
Outlier removal generates 4 lof-voted candidates
score-using-embedding Time: 1.9858300685882568s
compute-tf-idf-class_count Time: 2.385061025619507s
compute-tf-idf-property_count Time: 2.877363920211792s


2it [00:20, 10.06s/it]

...

291it [33:50,  6.97s/it]

mosaic-features Time: 0.009000301361083984s
Qnodes to lookup: 2390
Qnodes from file: 2362
Column_vector_stragtegy centroid_of_lof failed
score-using-embedding Time: 1.2583339214324951s
compute-tf-idf-class_count Time: 1.5948200225830078s
compute-tf-idf-property_count Time: 1.844005823135376s


292it [33:56,  6.86s/it]

mosaic-features Time: 0.024094104766845703s
Qnodes to lookup: 2447
Qnodes from file: 2430
Outlier removal generates 2 lof-voted candidates
Outlier removal generates 4 lof-voted candidates
Column_vector_stragtegy centroid_of_lof failed
Outlier removal generates 5 lof-voted candidates
score-using-embedding Time: 1.7655959129333496s
compute-tf-idf-class_count Time: 3.2045528888702393s
compute-tf-idf-property_count Time: 3.0077178478240967s


293it [34:08,  6.99s/it]


In [7]:
features_3(dev_features3_path, dev_feature_path, dev_graph_embedding, dev_class_count, dev_prop_count)

0it [00:00, ?it/s]

mosaic-features Time: 0.0163271427154541s
Qnodes to lookup: 3083
Qnodes from file: 3016
Outlier removal generates 5 lof-voted candidates
Outlier removal generates 2 lof-voted candidates
Outlier removal generates 2 lof-voted candidates
score-using-embedding Time: 1.540473222732544s
compute-tf-idf-class_count Time: 2.2249600887298584s
compute-tf-idf-property_count Time: 3.083150863647461s


1it [00:10, 10.58s/it]

mosaic-features Time: 0.00791621208190918s
Qnodes to lookup: 1770
Qnodes from file: 1746
Outlier removal generates 6 lof-voted candidates
score-using-embedding Time: 0.8021562099456787s
compute-tf-idf-class_count Time: 1.153777837753296s
compute-tf-idf-property_count Time: 1.372708797454834s


2it [00:16,  8.03s/it]

...

49it [05:40,  6.72s/it]

mosaic-features Time: 0.00503230094909668s
Qnodes to lookup: 1745
Qnodes from file: 1732
Outlier removal generates 4 lof-voted candidates
score-using-embedding Time: 0.7070739269256592s
compute-tf-idf-class_count Time: 1.0511889457702637s
compute-tf-idf-property_count Time: 1.236907958984375s


50it [05:45,  6.16s/it]

mosaic-features Time: 0.006414175033569336s
Qnodes to lookup: 2305
Qnodes from file: 2248
Outlier removal generates 5 lof-voted candidates
score-using-embedding Time: 0.7984271049499512s
compute-tf-idf-class_count Time: 1.1640949249267578s
compute-tf-idf-property_count Time: 1.3746988773345947s


51it [05:50,  6.87s/it]


### Generate Training Data

In [34]:
features = [
            "monge_elkan","monge_elkan_aliases","jaro_winkler",
            "levenshtein","singleton","pgr_rts","context_score_3",
            "pgt_centroid_score","pgt_class_count_tf_idf_score",
            "pgt_property_count_tf_idf_score", "num_tokens", "num_char"
            ]

In [35]:
def merge_files(args):
    datapath = args.train_path
    df_list  = []
    for fn in glob.glob(f"{datapath}/*csv"):
        if os.path.getsize(fn) == 0:
            continue
        fid = fn.split('/')[-1][:-4]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df['context_score'].fillna(0.0, inplace=True)
        df_list.append(df)            
    return pd.concat(df_list) 

def compute_normalization_factor(args, all_data):
    min_max_scaler_path = args.min_max_scaler_path
    all_data_features = all_data[features]
    scaler = MinMaxScaler()
    scaler.fit(all_data_features)
    pickle.dump(scaler, open(min_max_scaler_path, 'wb'))
    return scaler

In [36]:
def generate_train_data(args):
    scaler_path = args.min_max_scaler_path
    scaler = pickle.load(open(scaler_path, 'rb'))
    final_list = []
    sfeatures = copy.deepcopy(features) + ['evaluation_label']
    normalize_features = features
    evaluation_label = ['evaluation_label']
    positive_features_final = []
    negative_features_final = []
    for i,file in enumerate(glob.glob(args.train_path + '/*.csv')):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
                continue
        d_sample = pd.read_csv(file)
#         d_sample['context_score'].fillna(0.0, inplace=True)
        grouped_obj = d_sample.groupby(['column', 'row'])
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            pos_features = []
            neg_features = []
            a = cell[1][cell[1]['evaluation_label'] == 1]
            if a.empty:
                continue
            pos_rows = cell[1][cell[1]['evaluation_label'].astype(int) == 1][features].to_numpy()
            for i in range(len(pos_rows)):
                pos_features.append(pos_rows[i])
            neg_rows = cell[1][cell[1]['evaluation_label'].astype(int) == -1][features].to_numpy()
            for i in range(min(20,len(neg_rows))):
                neg_features.append(neg_rows[i])
            random.shuffle(pos_features)
            random.shuffle(neg_features)
            positive_features_final.append(pos_features)
            negative_features_final.append(neg_features)
            
    print(len(positive_features_final), len(positive_features_final[3]))
    print(len(negative_features_final), len(negative_features_final[3]))
    pickle.dump(positive_features_final,open(args.pos_output,'wb'))
    pickle.dump(negative_features_final,open(args.neg_output,'wb'))

In [37]:
gen_training_data_args = Namespace(train_path=train_feature_path, pos_output=pos_output, neg_output=neg_output, 
                 min_max_scaler_path=min_max_scaler_path)
all_data = merge_files(gen_training_data_args)
scaler = compute_normalization_factor(gen_training_data_args, all_data)

/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/1529062613.py:3: DtypeWarning: Columns (30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  all_data = merge_files(gen_training_data_args)
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/1529062613.py:3: DtypeWarning: Columns (36,38,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  all_data = merge_files(gen_training_data_args)


In [38]:
len(all_data)

1715832

In [39]:
all_data.head()

,column,row,label,context,filename,column-id,label_clean,kg_id,kg_labels,kg_aliases,...,context_score_3,num_char,num_tokens,is_lof,pgt_centroid_score,pgt_class_count_tf_idf_score,top5_class_count,pgt_property_count_tf_idf_score,top5_property_count,table_id
0,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q4533523,Yuvileine|Juwilejne,NaN,...,0.8501,19,1,-1,0.871381,0.934188,Q7216840:0.107|Q2989457:0.094|Q203323:0.069|Q1...,0.621857,P1077:0.084|P94:0.059|P2046:0.054|P1082:0.053|...,042AKDN1
1,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q4533526,Yuvileine|Juwilejne,NaN,...,0.1498,19,1,-1,0.824404,0.397589,Q486972:0.028|Q177634:0.028|Q874405:0.027|Q788...,0.386139,P1077:0.084|P1376:0.078|P6766:0.052|P2044:0.04...,042AKDN1
2,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q1011429,Iouvileïne|Sloboschanske|Slobozhanske,Yuvileine|Iouvileine|Juwilejne,...,0.1356,37,1,-1,0.871398,0.934188,Q7216840:0.107|Q2989457:0.094|Q203323:0.069|Q1...,0.517515,P1077:0.084|P1376:0.078|P94:0.059|P1082:0.053|...,042AKDN1
3,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q4533526,Yuvileine|Juwilejne,NaN,...,0.1498,19,1,-1,0.824404,0.397589,Q486972:0.028|Q177634:0.028|Q874405:0.027|Q788...,0.386139,P1077:0.084|P1376:0.078|P6766:0.052|P2044:0.04...,042AKDN1
4,0,0,Yuvileine,15830|84|2.83,042AKDN1.csv,042AKDN1.csv-0,Yuvileine,Q1011429,Sloboschanske|Iouvileïne|Slobozhanske,Yuvileine|Juwilejne|Iouvileine,...,0.1356,37,1,-1,0.871398,0.934188,Q7216840:0.107|Q2989457:0.094|Q203323:0.069|Q1...,0.517515,P1077:0.084|P1376:0.078|P94:0.059|P1082:0.053|...,042AKDN1


In [40]:
generate_train_data(gen_training_data_args)

/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3563378694.py:1: DtypeWarning: Columns (30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_train_data(gen_training_data_args)
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3563378694.py:1: DtypeWarning: Columns (30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_train_data(gen_training_data_args)
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3563378694.py:1: DtypeWarning: Columns (36,38,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  generate_train_data(gen_training_data_args)


12542 3
12542 20


### Model Definition

In [41]:
# Dataset
class T2DV2Dataset(Dataset):
    def __init__(self, pos_features, neg_features):
        self.pos_features = pos_features
        self.neg_features = neg_features
    
    def __len__(self):
        return len(self.pos_features)
    
    def __getitem__(self, idx):
        return self.pos_features[idx], self.neg_features[idx]

# Model
class PairwiseNetwork(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        #original 12x24, 24x12, 12x12, 12x1
        self.fc1 = nn.Linear(hidden_size, 2*hidden_size)
        self.fc2 = nn.Linear(2*hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
    
    def forward(self, pos_features, neg_features):
        # Positive pass
        x = F.relu(self.fc1(pos_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pos_out = torch.sigmoid(self.fc4(x))
        
        # Negative Pass
        x = F.relu(self.fc1(neg_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        neg_out = torch.sigmoid(self.fc4(x))
        
        return pos_out, neg_out
    
    def predict(self, test_feat):
        x = F.relu(self.fc1(test_feat))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        test_out = torch.sigmoid(self.fc4(x))
        return test_out

# Pairwise Loss
class PairwiseLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.m = 0
    
    def forward(self, pos_out, neg_out):
        distance = (1 - pos_out) + neg_out
        loss = torch.mean(torch.max(torch.tensor(0), distance))
        return loss

### Training

In [42]:
def generate_dataloader(positive_feat_path, negative_feat_path):
    pos_features = pickle.load(open(positive_feat_path, 'rb'))
    neg_features = pickle.load(open(negative_feat_path, 'rb'))

    pos_features_flatten = list(chain.from_iterable(pos_features))
    neg_features_flatten = list(chain.from_iterable(neg_features))

    train_dataset = T2DV2Dataset(pos_features_flatten, neg_features_flatten)
    train_dataloader = DataLoader(train_dataset, batch_size=64)
    return train_dataloader

def infer_scores(min_max_scaler_path, input_table_path, output_table_path, model):
    scaler = pickle.load(open(min_max_scaler_path, 'rb'))
    normalize_features = features
    for file in glob.glob(input_table_path + '/*.csv'):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
                continue
                
        d_sample = pd.read_csv(file)
#         d_sample['context_score'].fillna(0.0, inplace=True)
        grouped_obj = d_sample.groupby(['column', 'row'])
        new_df_list = []
        pred = []
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            sorted_df = cell[1].sort_values('context_score', ascending=False)
            sorted_df_features = sorted_df[normalize_features]
            new_df_list.append(sorted_df)
            arr = sorted_df_features.to_numpy()
            test_inp = []
            for a in arr:
                test_inp.append(a)
            test_tensor = torch.tensor(test_inp).float()
            scores = model.predict(test_tensor)
            scores_list = torch.squeeze(scores).tolist()
            if not type(scores_list) is list:
                pred.append(scores_list)
            else:
                pred.extend(scores_list)
        test_df = pd.concat(new_df_list)
        test_df[final_score_column] = pred
        test_df.to_csv(f"{output_table_path}/{file_name}", index=False)

def train(args):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    
    else:
        device = torch.device('cpu')
    train_dataloader = generate_dataloader(args.positive_feat_path, args.negative_feat_path)
    criterion = PairwiseLoss()
    EPOCHS = args.num_epochs
    model = PairwiseNetwork(len(features)).to(device=device)
    optimizer = Adam(model.parameters(), lr=args.lr)
    top1_max_prec = 0
    for epoch in range(EPOCHS):
        train_epoch_loss = 0
        avg_loss = 0
        model.train()
        for bid, batch in tqdm(enumerate(train_dataloader), position=0, leave=True):
            positive_feat = torch.tensor(batch[0].float())
            negative_feat = torch.tensor(batch[1].float())
            optimizer.zero_grad()
            pos_out, neg_out = model(positive_feat, negative_feat)
            loss = criterion(pos_out, neg_out)
            loss.backward()
            optimizer.step()
            train_epoch_loss += loss
        avg_loss = train_epoch_loss / bid

        # Evaluation
        model.eval()
        infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
        eval_data = merge_eval_files(args.dev_output)
        res, candidate_eval_data = parse_eval_files_stats(eval_data, final_score_column)
        top1_precision = res['num_tasks_with_model_score_top_one_accurate']/res['num_tasks_with_gt']
        if top1_precision > top1_max_prec:
            top1_max_prec = top1_precision
            model_save_name = 'epoch_{}_loss_{}_top1_{}.pth'.format(epoch, avg_loss, top1_max_prec)
            best_model_path = os.path.join(args.model_save_path, model_save_name)
            torch.save(model.state_dict(), best_model_path)
        
        print("Epoch {}, Avg Loss is {}, epoch top1 {}, max top1 {}".format(epoch, avg_loss, top1_precision, top1_max_prec))
    return best_model_path

In [43]:
def merge_eval_files(final_score_path):
    eval_file_names = []
    df_list = []
    for (dirpath, dirnames, filenames) in os.walk(final_score_path):
        for fn in filenames:
            if "csv" not in fn:
                continue
            abs_fn = os.path.join(dirpath, fn)
            assert os.path.isfile(abs_fn)
            if os.path.getsize(abs_fn) == 0:
                continue
            eval_file_names.append(abs_fn)
    
    for fn in eval_file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df_list.append(df)
    return pd.concat(df_list)

def parse_eval_files_stats(eval_data, method):
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'column', 'row'])['table_id'].count().reset_index(name="count")
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'column', 'row']))
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    has_gt_list = []
    has_gt_in_candidate = []
    for i, row in candidate_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
                    
        #rank on model score
        s_data = c_e_data.sort_values(by=[method], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)
            
    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    return res, candidate_eval_data

In [44]:
training_args = Namespace(num_epochs=20, lr=0.001, positive_feat_path=pos_output, negative_feat_path=neg_output,
                         dev_path=dev_feature_path, dev_output=dev_output_predictions,
                         model_save_path=model_save_path, min_max_scaler_path=min_max_scaler_path)

In [45]:
## Call Training
best_model_path = train(training_args)

0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
532it [00:00, 761.99it/s]


Epoch 0, Avg Loss is 0.4233013391494751, epoch top1 0.9461606354810238, max top1 0.9461606354810238


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
532it [00:00, 804.38it/s]


Epoch 1, Avg Loss is 0.14832746982574463, epoch top1 0.9496910856134158, max top1 0.9496910856134158


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
532it [00:00, 761.94it/s]


...

Epoch 17, Avg Loss is 0.11518098413944244, epoch top1 0.9492497793468667, max top1 0.9545454545454546


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
532it [00:00, 678.07it/s]


Epoch 18, Avg Loss is 0.11511637270450592, epoch top1 0.9488084730803178, max top1 0.9545454545454546


0it [00:00, ?it/s]/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
/var/folders/qv/cxzpwz3j29x7n79vwpw253v80000gn/T/ipykernel_22238/3986128095.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
532it [00:00, 654.82it/s]


Epoch 19, Avg Loss is 0.11449259519577026, epoch top1 0.9479258605472197, max top1 0.9545454545454546


In [46]:
best_model_path

'/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/tbl-pipeline-nn/v19/saved_models/epoch_5_loss_0.12551966309547424_top1_0.9545454545454546.pth'

## Dev Prediction

In [57]:
def dev_prediction(dev_feature_path, dev_predictions_top_k, saved_model, output_column, min_max_scaler_path, k=5):
    for file in glob.glob(dev_feature_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        feature_str =  ",".join(features)
        if os.path.getsize(file) == 0:
            continue
        # location where the output generated by the predictions wil be stored.
        dev_output = f"{dev_predictions_top_k}/{filename}"
        !tl --log-file $tl_log_file predict-using-model $file -o $output_column \
            --features $feature_str \
            --ranking-model $saved_model \
            --normalization-factor $min_max_scaler_path \
            / create-pseudo-gt \
            --column-thresholds $threshold \
            --filter smc_class_score:0 \
            / get-kg-links -c $final_score_column -k $k --k-rows \
            > $dev_output

In [58]:
def add_color(dev_predictions_top_k, dev_colorized_path, score_column, k=5):
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
                
        dev_color_file = f"{dev_colorized_path}/{filename[:-4]}.xlsx"
        !tl add-color $file -c "$score_column,evaluation_label" -k $k --output $dev_color_file

In [59]:
def compute_metrics(dev_predictions_top_k, dev_predictions_metrics, score_column, k=5):
    df_list = []
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
                
        dev_metrics_file = f"{dev_predictions_metrics}/{filename}"
        !tl --log-file $tl_log_file metrics $file -k $k -c $score_column --tag $filename> $dev_metrics_file
        df_list.append(pd.read_csv(dev_metrics_file))
    return pd.concat(df_list)

In [60]:
def compute_custom_metrics(dev_predictions_top_k):
    df_list = []
    for file in glob.glob(dev_predictions_top_k+"/*.csv"):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        col_df = []
        for col, coldf in df.groupby(by=["column"]):
            rows = 0
            pgt_rows = 0
            pgt_recall = 0
            unignored_rows = 0
            unignored_candidates = 0
            unignored_correct = 0
            ignored_correct = 0
            kth_perc_rows = 0
            kth_perc_correct = 0
            kth_perc_candidates = 0
            for row, rowdf in coldf.groupby(by=["row"]):
                rows += 1
                p_count = rowdf[(rowdf["pseudo_gt"] == 1)].shape[0]
                if p_count > 0:
                    pgt_rows += 1
                p_recall = rowdf[((rowdf["pseudo_gt"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
                pgt_recall += p_recall
                ignore_0_count = rowdf[rowdf["ignore_candidate"] == 0].shape[0]
                if ignore_0_count > 0:
                    unignored_rows += 1
                unignored_candidates += ignore_0_count
                unignored_correct += rowdf[((rowdf["ignore_candidate"] == 0) & (rowdf["evaluation_label"] == 1))].shape[0]
                ignored_correct += rowdf[((rowdf["ignore_candidate"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
                kth_perc_1_count = rowdf[rowdf["kth_percenter"] == 1].shape[0]
                if kth_perc_1_count > 0:
                    kth_perc_rows += 1
                kth_perc_candidates += kth_perc_1_count
                kth_perc_correct += rowdf[((rowdf["kth_percenter"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
            unique_rows = coldf["label"].nunique()
            col_df.append(pd.DataFrame([{
                "filename":filename,
                "column": col,
                "rows": rows,
                "unique_rows": unique_rows,
                "pgt_rows": pgt_rows,
                "pgt_recall": pgt_recall,
                "pgt_accuracy": pgt_recall/pgt_rows if pgt_rows!=0 else 0,
                "unignored_rows": unignored_rows,
                "unignored_candidates": unignored_candidates,
                "unignored_correct": unignored_correct,
                "ignored_correct": ignored_correct,
                "ignore_candidate_accuracy": unignored_correct/unignored_rows if unignored_rows != 0 else 0,
                "kth_percenter_rows": kth_perc_rows,
                "kth_percenter_candidates": kth_perc_candidates,
                "kth_percenter_correct": kth_perc_correct,
                "kth_percenter_accuracy": kth_perc_correct/kth_perc_rows
            }]))
        df_list.append(pd.concat(col_df))
    return pd.concat(df_list)

In [61]:
print(dev_feature_path, dev_predictions_top_k, best_model_path, final_score_column, min_max_scaler_path)

/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/tbl-pipeline-nn/v19/features /Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/tbl-pipeline-nn/v19/dev_predictions_top_k /Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/tbl-pipeline-nn/v19/saved_models/epoch_5_loss_0.12551966309547424_top1_0.9545454545454546.pth siamese_prediction /Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/temp/training_data/tl_pipeline_normalization_factor.pkl


In [62]:
dev_prediction(dev_feature_path, dev_predictions_top_k, best_model_path, final_score_column, min_max_scaler_path, k=5)

4PKEEJU4.csv
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyW

In [63]:
metrics_df = compute_metrics(dev_predictions_top_k, dev_metrics_path, final_score_column, k=1)

In [64]:
metrics_df

,k,f1,precision,recall,tag
0,1,1.000000,1.000000,1.000000,4PKEEJU4.csv
0,1,1.000000,1.000000,1.000000,4DC5O5I4.csv
0,1,0.908523,0.931818,0.886364,58E7A5WL.csv
0,1,0.966667,0.966667,0.966667,0FQOOJPU.csv
0,1,1.000000,1.000000,1.000000,5IXA0RAI.csv
0,1,1.000000,1.000000,1.000000,0LF7RI6N.csv
0,1,0.887500,0.887500,0.887500,2PKG4E2V.csv
0,1,0.949153,0.949153,0.949153,50NWQJ1T.csv
0,1,0.883333,0.883333,0.883333,44NDHWR1.csv
0,1,0.983333,0.983333,0.983333,39W7XXTI.csv


In [65]:
metrics_df.to_csv(f"{dev_metrics_path}/metrics_1.csv", index=False)

In [66]:
add_color(dev_predictions_top_k, dev_colorized_path, final_score_column)

4PKEEJU4.csv
add-color Time: 0.1495048999786377s
4DC5O5I4.csv
add-color Time: 0.07369184494018555s
58E7A5WL.csv
add-color Time: 0.10478591918945312s
0FQOOJPU.csv
add-color Time: 0.12833833694458008s
5IXA0RAI.csv
add-color Time: 0.08734393119812012s
0LF7RI6N.csv
add-color Time: 0.11375308036804199s
2PKG4E2V.csv
add-color Time: 0.1915898323059082s
50NWQJ1T.csv
add-color Time: 0.15299081802368164s
44NDHWR1.csv
add-color Time: 0.14342904090881348s
39W7XXTI.csv
add-color Time: 0.14214229583740234s
3WXFYEAX.csv
add-color Time: 0.08128881454467773s
28D7RBJT.csv
add-color Time: 0.10819411277770996s
1GUVMENF.csv
add-color Time: 0.08947014808654785s
0QWF60VG.csv
add-color Time: 0.14881682395935059s
3OCW1LDZ.csv
add-color Time: 0.1747448444366455s
4V4O0CTS.csv
add-color Time: 0.13443398475646973s
29D1VZHF.csv
add-color Time: 0.2166440486907959s
4FG1UN8O.csv
add-color Time: 0.23493123054504395s
1ZFRQBQS.csv
add-color Time: 0.3665320873260498s
0WBTX8LY.csv
add-color Time: 0.13002300262451172s
1NS33

In [67]:
def find_missing_correct_candidates(candidates_path, missing_correct_candidates_path):
     for file in tqdm(glob.glob(candidates_path + '/*.csv')):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
        missing_file = f"{missing_correct_candidates_path}/{filename}"
        !tl check-candidates "$file" > "$missing_file"

In [68]:
find_missing_correct_candidates(dev_candidate_path, dev_missing_candidates_path)

  0%|                                                                                                                                                                                                                         | 0/51 [00:00<?, ?it/s]

check-candidates Time: 0.013679027557373047s


  2%|████                                                                                                                                                                                                             | 1/51 [00:03<02:32,  3.05s/it]

check-candidates Time: 0.007055997848510742s


  4%|████████▏                                                                                                                                                                                                        | 2/51 [00:07<03:20,  4.10s/it]

...

 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊        | 49/51 [03:05<00:08,  4.04s/it]

check-candidates Time: 0.009033918380737305s


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉    | 50/51 [03:08<00:03,  3.66s/it]

check-candidates Time: 0.010251998901367188s


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [03:11<00:00,  3.75s/it]


In [69]:
def concat_files(files_path):
    df_list = []
    for file in glob.glob(files_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        df['filename'] = filename
        df_list.append(df)
    return pd.concat(df_list)

In [70]:
missing_df = concat_files(dev_missing_candidates_path)

4PKEEJU4.csv
4DC5O5I4.csv
58E7A5WL.csv
0FQOOJPU.csv
5IXA0RAI.csv
0LF7RI6N.csv
2PKG4E2V.csv
50NWQJ1T.csv
44NDHWR1.csv
39W7XXTI.csv
3WXFYEAX.csv
28D7RBJT.csv
1GUVMENF.csv
0QWF60VG.csv
3OCW1LDZ.csv
4V4O0CTS.csv
29D1VZHF.csv
4FG1UN8O.csv
0LZ0M8W4.csv
3B54GZSX.csv
1ZFRQBQS.csv
0WBTX8LY.csv
1NS33P8C.csv
2TGNKH1P.csv
53OUTCE4.csv
5TJI4XTK.csv
4DPRZWVL.csv
4SOL8H0M.csv
080HU8A5.csv
3OAZEVOY.csv
13BLTPJD.csv
3M5QXPWN.csv
0G9YPQC0.csv
1YPLVLS9.csv
0CETTKTA.csv
1GOKLC0K.csv
2QFYH2N9.csv
00ECUL14.csv
2E6QBLCA.csv
1T91CHXV.csv
59W76Q0Y.csv
5PKTGQ6Q.csv
3JNFST2K.csv
1XIWQBSF.csv
2YCSL7OH.csv
2FSRG0OI.csv
0P8H49LQ.csv
2FXR6BX7.csv
1PTL0CX1.csv
0TQXSY28.csv
0JSF530F.csv


In [71]:
missing_df

,column,row,label,context,GT_kg_id,GT_kg_label,filename
0,0,0,V* EM And,22.0593421416298|Mira variable|Andromeda,Q74157334,EM Andromedae,0FQOOJPU.csv
1,0,1,V* BB And,4.4331385872306|Mira variable|Andromeda,Q74006182,BB Andromedae,0FQOOJPU.csv
0,2,13,Phoenix,Forever Living Products|food industry|Rex Maughan,Q49221,Scottsdale,2PKG4E2V.csv
1,3,17,Amaury Vergara,Saulo y Yairo|food industry|Guadalajara,Q729642,Jorge Vergara,2PKG4E2V.csv
0,0,12,ankin Bridge,Union Railroad Port Perry Bridge|Carrie Furnac...,Q14629009,Rankin Bridge,39W7XXTI.csv
0,0,13,Hreising,13461.024|8901.398,Q10497,Freising,3WXFYEAX.csv
1,0,15,assau,17498.364|9386.19,Q10489,Passau,3WXFYEAX.csv
0,0,2,Cirzeit,4574.29|13.972|811.74,Q212381,Birzeit,1GUVMENF.csv
1,0,4,ouler,108738.54|66.410786144|1646.725,Q192517,Boulder,1GUVMENF.csv
2,0,11,Dremen,564797.184|325.52764|5.952,Q24879,Bremen,1GUVMENF.csv


In [72]:
missing_df.to_csv(f"{dev_missing_candidates_path}/missing_concatenated.csv", index=False)

In [74]:
find_missing_correct_candidates(train_candidate_path, train_missing_candidates_path)

  0%|                                                                                                                                                                                                                        | 0/301 [00:00<?, ?it/s]

check-candidates Time: 0.007332801818847656s


  0%|▋                                                                                                                                                                                                               | 1/301 [00:03<18:18,  3.66s/it]

check-candidates Time: 0.008497953414916992s


  1%|█▍                                                                                                                                                                                                              | 2/301 [00:08<21:55,  4.40s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 299/301 [14:08<00:04,  2.09s/it]

check-candidates Time: 0.007487773895263672s


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 300/301 [14:09<00:02,  2.05s/it]

check-candidates Time: 0.017284870147705078s


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 301/301 [14:11<00:00,  2.83s/it]


In [75]:
train_missing_df = concat_files(train_missing_candidates_path)

042AKDN1.csv
1DQVRVUL.csv
2VWPPCQM.csv
4IJHCIR9.csv
2OYAB7YM.csv
5894JWA5.csv
4Q6FAT24.csv
3S6WON79.csv
4IB27COT.csv
2S72D3R9.csv
49K301SA.csv
03XJOJ15.csv
0KL64BZL.csv
1XV2X3XE.csv
2J3MZCRD.csv
59CFTDPL.csv
19WCLMKZ.csv
4722C9EQ.csv
59JNC864.csv
4KDMZHXJ.csv
17DA0MJ4.csv
2WRDGDBJ.csv
4UUFDEP4.csv
0VDX10IJ.csv
1QGWWYAF.csv
0IUSS6UH.csv
42BKS5LP.csv
40DGQ0R6.csv
21P58N2X.csv
10WA15QW.csv
4ENVC6F0.csv
39DC3Z7N.csv
302H92MV.csv
5VD9H3AD.csv
1AXBOBYJ.csv
4VL4BFPM.csv
0Z8LKW3C.csv
0KWK2FSC.csv
28Y667AR.csv
3K0JGMP1.csv
1A51CWBN.csv
4UZA5UWB.csv
15CM535P.csv
3CSAH6ZY.csv
0MYK9SQC.csv
06UFHM3B.csv
06ZS1QBE.csv
14DACNEL.csv
2OJXQ9F3.csv
0HZG8QGJ.csv
24OGNVVP.csv
0XWQ7DA6.csv
0ADP8WGU.csv
0P4Y07PN.csv
4C37XZHJ.csv
26CIDK81.csv
5J25NATF.csv
3AK02Q9B.csv
3EL4J49W.csv
1JY71MSY.csv
4LFJR3YN.csv
243JI1XC.csv
3SQ9UAK1.csv
258AFYQJ.csv
3HZEPYNC.csv
0ZZFENJS.csv
1XK9O8CF.csv
20PYU63R.csv
471N0AUD.csv
03Q5JG9I.csv
4N6X42PJ.csv
2FHCJAMA.csv
2ZYEX9U9.csv
4E99I3P7.csv
0VGKU21T.csv
4VAYQOG0.csv
1FIFDPBT.csv

In [76]:
len(train_missing_df)

109

In [77]:
train_missing_df

,column,row,label,context,GT_kg_id,GT_kg_label,filename
0,0,16,ouchon,United States of America|Thomas Keller,Q61969122,Bouchon,2VWPPCQM.csv
0,0,3,JD 2194,-46.207207|-0.16881000000000002,Q12067892,HD 23194,5894JWA5.csv
1,0,7,JD 22234,61.28388|-0.88644,Q12067734,HD 222349,5894JWA5.csv
0,0,11,Hope,Mountain Time Zone|1245,Q2525524,Hope,3S6WON79.csv
0,0,0,[FK2004] 118,44.74431|0.024648029999999998|160.17874999999998,Q78536288,[FK2002] 118,2J3MZCRD.csv
...,...,...,...,...,...,...,...
3,2,41,Kassrine,Arch of Alxander Severus|listed monument of Tu...,Q276576,Beja Governorate,0VC79Y6X.csv
0,0,7,nežino,Central European Time|5.952,Q1522139,Knežino,5739HKRB.csv
0,0,2,John,House of Plantagenet|John Lackland|Beaumont Pa...,Q129308,"John, King of England",3IB64ASX.csv
0,1,12,Turkestan,Turkistan|Brockhaus and Efron Encyclopedic Dic...,Q6549315,Turkestan,471ODBDH.csv


In [78]:
train_missing_df.to_csv(f"{train_missing_candidates_path}/missing_concatenated.csv", index=False)